In [1]:
!pip install torch torchvision torchaudio
!torch install easyocr

zsh:1: command not found: torch


In [2]:
import easyocr

In [3]:
import cv2
import os
import pandas as pd

from matplotlib import pyplot as plt

# Initialize the camera
cam = cv2.VideoCapture(0)

# Load the Haar cascade file for number plate detection
harcascade = 'haarcascade_russian_plate_number.xml'
plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + harcascade)

# Minimum area for the number plate to be considered
min_area = 500
count = 0

# Directory to save images
image_directory = './scanned_images/'
if not os.path.exists(image_directory):
    os.makedirs(image_directory)

while True:
    success, img = cam.read()
    
    if not success:
        print("Failed to capture image")
        break
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)
    
    for (x, y, w, h) in plates:
        area = w * h
        
        if area > min_area:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)
            
            img_roi = img[y: y+h, x:x+w]
            cv2.imshow("ROI", img_roi)
            
    cv2.imshow("Result", img)
    
    if cv2.waitKey(1) & 0xFF == ord('s'):
        image_path = os.path.join(image_directory, "scanned_image" + str(count) + ".jpg")
        cv2.imwrite(image_path, img_roi)
        cv2.rectangle(img, (0, 200), (640, 300), (255, 0, 0), cv2.FILLED)
        cv2.putText(img, "Number Plate Saved", (150, 265), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255), 2)
        cv2.imshow("Results", img)
        cv2.waitKey(500)
        count += 1
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cam.release()
cv2.destroyAllWindows()

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# DataFrame to store the results
results = pd.DataFrame(columns=["Image", "Number Plate"])

# Initialize an empty list to store DataFrames
dfs_to_concat = []

# Iterate over saved images
for file in os.listdir(image_directory):
    if file.startswith('scanned_image') and file.endswith('.jpg'):
        # Load the image
        img_path = os.path.join(image_directory, file)
        img = cv2.imread(img_path)
        
        # Perform OCR on the image
        result = reader.readtext(img)
        plate_number = ""
        if result:
            plate_number = result[0][-2]
            print(f"Detected Number Plate for {file}: {plate_number}")
            
            # Create a DataFrame representing the new row
            new_row = pd.DataFrame({"Image": [file], "Number Plate": [plate_number]})
            
            # Append the new DataFrame to the list
            dfs_to_concat.append(new_row)

# Concatenate all DataFrames in the list
results = pd.concat(dfs_to_concat, ignore_index=True)

# Save the results to an Excel file
results.to_excel("number_plate_results.xlsx", index=False)

Detected Number Plate for scanned_image2.jpg: SPDSO
Detected Number Plate for scanned_image3.jpg: R
Detected Number Plate for scanned_image1.jpg: EOP.807
Detected Number Plate for scanned_image0.jpg: EOP-807
Detected Number Plate for scanned_image4.jpg: ra
